In [ ]:
from jupyter_innotater import *
import numpy as np, os

## Save button calls your supplied Python function

In [ ]:
foodfns = sorted(os.listdir('./foods/'))
targets = np.zeros((len(foodfns), 4), dtype='int') # (x,y,w,h) for each data row

def my_save_hook(uindexes):
    np.savetxt("foodboxes.csv", targets, delimiter=",", fmt="%d")
    return True # Tell Innotater the save was successful (we just assume so here...)

In [ ]:
Innotater( ImageInnotation(foodfns, path='./foods'), BoundingBoxInnotation(targets), save_hook=my_save_hook )

Click the Save button above after making changes, and a csv file will be saved containing your latest data.

Your function should return True if the save was successful, otherwise False if the data should still be saved.

The uindexes parameter is a list of integers telling you which indexes have been changed through the Innotater.

## Custom Buttons calling your own Python function

The ButtonInnotation allows you to provide custom button functionality.

In this example, there is a button to reset everything in the current sample, and buttons to reset each bounding box.

In [ ]:
animalfns = sorted(os.listdir('./animals/'))

repeats = 8

# Per-photo data
classes = ['cat', 'dog']
targets_type = np.zeros((len(animalfns), len(classes)), dtype='int') # One-hot encoding

# Repeats within each photo
targets_bboxes = np.zeros((len(animalfns), repeats, 4), dtype='int') # (x,y,w,h) for each animal

def reset_click(uindex, repeat_index, **kwargs):
    # uindex is the (underlying) index of the data sample where the button was clicked
    # repeat_index will be the sub-index of the row in a RepeatInnotation, or -1 if at the top level
    # kwargs will contain name and desc fields

    if repeat_index == -1: # This was a top-level button (no sub-index within the RepeatInnotation)
        # So reset everything
        targets_type[uindex] = [1,0]
        for i in range(repeats):
            targets_bboxes[uindex, i, :] = 0
    else:
        # Only reset the row with repeat_index
        targets_bboxes[uindex, repeat_index, :] = 0
        
    return True # Tell Innotater the data at uindex was changed

Innotater(
    ImageInnotation(animalfns, path='./animals', width=400, height=300),
    [
        MultiClassInnotation(targets_type, name='Animal Type', classes=classes, dropdown=False),

        RepeatInnotation(
            (ButtonInnotation, None, {'desc': 'X', 'on_click': reset_click, 'layout': {'width': '40px'}}),
            (BoundingBoxInnotation, targets_bboxes),
             max_repeats=repeats, min_repeats=1
        ),
        
        ButtonInnotation(None, name='Reset All', on_click=reset_click)
    ]
)